<div align='center'>
    <img src="https://www.raconteur.net/wp-content/uploads/2018/04/shutterstock_762930772.jpg" width=400/>
</div>

<div class="column" style="max-width:100%;"> 
    <img src="https://cdn-1.motorsport.com/images/mgl/Y99JQRbY/s800/red-bull-racing-logo-1.jpg" width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/Alpine/_jcr_content/logo.img.jpg/1673451453749.jpg" width=70>
    <img src="https://cdn-8.motorsport.com/images/mgl/YEQJgyLY/s800/mclaren-f1-team-logo-1.jpg" width=60>
    <img src="https://assets.turbologo.com/blog/en/2019/10/19084952/ferrari-logo-illustration-958x575.jpg" width=70 />
      <img src="https://cdn.sanity.io/images/fnx611yr/production/ff454bb2b4adb541b66477adcb49088f8de0fb4a-2400x2400.jpg?rect=0,572,2400,1256&w=1200&h=628&auto=format"/ width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/Mercedes/_jcr_content/logo.img.jpg/1582638425211.jpg" width=70/>
    <img src="https://diginomica.com/sites/default/files/images/2019-05/Haas-F1-logo.jpg" width=70>
    <img src="https://d3cm515ijfiu6w.cloudfront.net/wp-content/uploads/2021/01/01082241/Aston-Martin.jpg" width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/AlphaTauri/_jcr_content/logo.img.jpg/1582650557134.jpg" width=70>
      <img src="https://www.formula1.com/content/dam/fom-website/manual/teams/Sauber/Alfa_Romeo_Racing_logo.jpg"/ width=70>
</div>

In [1]:
%%html
<style>
@font-face {
  font-family: F1;
  src: url(https://www.formula1.com/etc/designs/fom-website/fonts/F1Regular/Formula1-Regular.ttf);
}

span{
  font-family: F1;
}

a{
  font-family: F1;
}

.nav-pills>li.active>a:focus {
    color: #ffffff;
    background-color: lightgray;
}

.container-fluid, .container-fluid h1 {
    font-family: F1;
    line-height: 1.7;
}

.container-fluid p {
    font-family: F1;
    color:black;
}

h1,h2,h3,h4,h5,h6,p, table {
  font-family: F1;
  color: black
}
</style>

In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import os
import seaborn as sns
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import plotly.graph_objects as go
pd.set_option('display.precision', 1)

In [3]:
#DATA CLEANING
races = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/races.csv')
circuits = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/circuits.csv')
constructor_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructor_standings.csv')
constructor_results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructor_results.csv')
drivers = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv')
constructors = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv')
lap_times = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/lap_times.csv')
status = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/status.csv')
driver_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/driver_standings.csv')
seasons = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/seasons.csv')
pit_stops = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv')
sprint_results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/sprint_results.csv')
results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/results.csv')
qualifying = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/qualifying.csv')

my_date = date.today()
#Helpers
def get_sec(time_str):
    """Get seconds from time."""
    if time_str != '\\N':
        m, s = time_str.split(':')
        return float(m) * 60 + float(s)

#Colors
names = ['ALB', 'ALO', 'DEV', 'VER', 'TSU', 'STR', 'SAR', 'SAI', 'RUS',
       'PIA', 'PER', 'OCO', 'NOR', 'MAG', 'LEC', 'HUL', 'HAM', 'GAS',
       'BOT', 'ZHO'] 
color_map = {
        'ALB': '#00A0DE',
        'SAR': '#00A0DE',
        'ZHO': '#A42134',
        'BOT': '#A42134',
        'TSU': '#20394C',
        'DEV': '#20394C',
        'VER': '#000B8D',
        'PER': '#000B8D',
        'HAM': '#00A19B',
        'RUS': '#00A19B',
        'MAG': '#E6002B',
        'HUL': '#E6002B', 
          'ALO': '#002420', 
          'STR': '#002420', 
          'LEC': '#EF1A2D',
          'SAI':'#EF1A2D',
          'OCO': '#2173B8',
          'GAS': '#2173B8',
        'NOR': '#FF8000' ,
        'PIA': '#FF8000'
}



races['date'] = pd.to_datetime(races['date'])
pit_stops['duration'] = pit_stops['milliseconds']/1000.0
lap_times['milliseconds'] = lap_times['milliseconds'].astype(float)
lap_times['duration'] = lap_times['milliseconds']/1000.0

results

races_23 = races[races['year'] == 2022]
# find first date in this table before current date
#last_race = races_23[races_23['date'] > pd.Timestamp.now().normalize()]
last_race = races_23[races_23['date'] < pd.Timestamp(2022,3,31)].sort_values(by='date').iloc[0]
current_raceid = last_race['raceId']
current_round = last_race['round']
current_circuit = last_race['circuitId']
current_race_name = last_race['name']
current_year = last_race['year']


#Driver by Lap
race_laps = lap_times[lap_times['raceId'] == current_raceid]
lap_driver = race_laps.merge(drivers, on='driverId', how='left', suffixes=('_1', '_2'))
lap_driver_no = lap_driver[(stats.zscore(lap_driver['duration'])) <1]



#RESULTS
result_race = results[results['raceId'] == current_raceid].copy()
num_laps = max(result_race['laps'])
result_race['lap'] = 0
res_race_laps = result_race.merge(drivers, on='driverId', how='left', suffixes=('_1', '_2'))
reduced = res_race_laps[['lap', 'grid', 'code']]
reduced.columns = ['lap', 'position', 'code']
reduced_res = pd.concat([reduced, lap_driver[['lap', 'position', 'code']]], axis=0)
res_race_laps = res_race_laps[res_race_laps['lap'] > 0]


#Track

track = circuits[circuits['circuitId'] == current_circuit]
track_lat = track['lat'].values[0]
track_lng = track['lng'].values[0]

#Podium
res_race_lap_status = result_race.merge(status, on='statusId', how='left', suffixes=('_1', '_2'))
res_race_lap_status = res_race_lap_status.merge(drivers, on='driverId', how='left', suffixes=('_1', '_2'))
res_race_lap_status_pod = res_race_lap_status[res_race_lap_status['position'] != '\\N'].copy()
res_race_lap_status_pod['position'] = res_race_lap_status_pod['position'].astype(int)
res_race_lap_status_pod = res_race_lap_status_pod.sort_values(by='position')



# Qualifying To Finishing path
qual_race = qualifying[qualifying['raceId'] == current_raceid]
qual_driver = qual_race.merge(drivers, on='driverId', how='left', suffixes=('_1', '_2'))
qual_driver['q1_dur'] = qual_driver['q1'].apply(lambda x: get_sec(x))
qual_driver['q2_dur'] = qual_driver['q2'].apply(lambda x: get_sec(x))
qual_driver['q3_dur'] = qual_driver['q3'].apply(lambda x: get_sec(x))
qual_driver['max_time'] = qual_driver.apply(lambda x: max(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
qual_driver['min_time'] = qual_driver.apply(lambda x: min(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
max_time = max(qual_driver['max_time'])
min_time = min(qual_driver['min_time']) - 1


#Pit Stops
race_pits = pit_stops[pit_stops['raceId']  == current_raceid]
# join with racer names
pit_driver = race_pits.merge(drivers, on='driverId', how='inner', suffixes=('_1', '_2'))
driver_group = pd.DataFrame()
mean = pit_driver.groupby(['code'])['duration'].mean()
mini = pit_driver.groupby(['code'])['duration'].min()
maxi = pit_driver.groupby(['code'])['duration'].max()
driver_group['mean'] = mean
driver_group['max'] = maxi
driver_group['min'] = mini
driver_group['code'] = driver_group.index
driver_group = driver_group.sort_values(by="max")

In [4]:
results.round(decimals=2).head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [5]:
# Styling
def podium(styler):
    styler.set_caption("Podium Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
   # styler.format({'points': "{.1f}"})
    return styler


def results_style(styler):
    styler.set_caption("Race Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def driver_stand(styler):
    styler.set_caption("Driver Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def const_standings(styler):
    styler.set_caption("Constructor Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlOrRd")
    styler.hide_index()
    return styler

In [6]:
display(md(f"# {current_race_name}, {current_year} Data Summary"))

# Bahrain Grand Prix, 2022 Data Summary

In [7]:
!pip install gmplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 383.4 kB/s eta 0:00:00


In [8]:
fig = px.scatter_mapbox(track, lat="lat", lon="lng",
                        color_discrete_sequence=["fuchsia"], zoom=12, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Podium

In [9]:
res_race_lap_status_pod[['code', 'grid', 'position', 'points']].iloc[:3].style.pipe(podium)

code,grid,position,points
LEC,1,1,26.0
SAI,3,2,18.0
HAM,5,3,15.0


# Full Results

In [10]:
res_race_lap_status = res_race_lap_status.sort_values(by='position')
res_race_lap_status_pod[['code', 'grid', 'position', 'points', 'status']].style.pipe(results_style)

code,grid,position,points,status
LEC,1,1,26.0,Finished
SAI,3,2,18.0,Finished
HAM,5,3,15.0,Finished
RUS,9,4,12.0,Finished
MAG,7,5,10.0,Finished
BOT,6,6,8.0,Finished
OCO,11,7,6.0,Finished
TSU,16,8,4.0,Finished
ALO,8,9,2.0,Finished
ZHO,15,10,1.0,Finished


# Qualifying

In [11]:
fig = go.Figure(layout_yaxis_range=[min_time, max_time])
fig.add_trace(go.Bar(
    x=qual_driver['code'],
    y=qual_driver['q1_dur'],
    name='Q1',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=qual_driver['code'],
    y=qual_driver['q2_dur'],
    name='Q2',
    marker_color='gold'
))
fig.add_trace(go.Bar(
    x=qual_driver['code'],
    y=qual_driver['q3_dur'],
    name='Q3',
    marker_color='orange'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, title="Qualifying Results")
fig.show()

In [12]:
#start/finish path
lap_driver_sf = reduced_res[(reduced_res['lap'] == 0) | (reduced_res['lap'] == num_laps)]

df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(lap_driver_sf, x="lap", y="position", color='code', color_discrete_map=color_map, title="Qualifying to Finish")
fig.show()

# Pit Stops

In [13]:
data = [
    go.Bar(y=driver_group['max'], x=driver_group.code, name='Max Pit',  
            #base=0
           ),
go.Bar(y=driver_group['mean'], x=driver_group.code, name='Avg PIt'),
go.Bar(y=driver_group['min'], x=driver_group.code, name='Min PIt')]

layout = go.Layout(
    barmode='overlay',
    title="Pit Times"
)

fig = dict(data = data, layout = layout, layout_yaxis_range=[23, 31])
from plotly.offline import init_notebook_mode, iplot
iplot(fig, show_link=False)

# Lap Times

In [14]:
my_layout = go.Layout({"title": "Views by publisher",
                       "yaxis": {"title":"Views"},
                       "xaxis": {"title":"Publisher"},
                       "showlegend": False})
fig = px.box(lap_driver_no, x="duration", y="code", color='code', points=False, color_discrete_map=color_map, labels={
    "code": "Driver",
    "duration":"Duration (Seconds)"
})
fig.update_layout(showlegend=False, title='Lap Times')
fig.show()

In [15]:
#remove outliers, use plotly
fig = px.line(lap_driver_no, x="lap", y="duration", color='code', labels={
    "duration": "Lap Time (Seconds)",
    "lap": "Lap"
}, title='Lap Times', color_discrete_map=color_map)
fig.show()

In [16]:
fig = px.line(lap_driver, x="lap", y="position", color='code', labels={
    "position":  "Position",
    "lap": "Lap"
}, title='Position By Lap', color_discrete_map=color_map)
fig.show()

# Driver Standings

In [17]:
drive_stand = driver_standings.merge(drivers, on='driverId', how='left', suffixes=('_1', '_2'))
drive_stand_race = drive_stand[drive_stand['raceId'] == (current_raceid)]
drive_stand_race  = drive_stand_race .sort_values(by='points', ascending=False)
drive_stand_race[['code', 'points']].style.pipe(driver_stand)

code,points
LEC,26.0
SAI,18.0
HAM,15.0
RUS,12.0
MAG,10.0
BOT,8.0
OCO,6.0
TSU,4.0
ALO,2.0
ZHO,1.0


# Constructor Standings

In [18]:
const_stand = constructor_standings.merge(constructors, on='constructorId', how='left', suffixes=('_1', '_2'))

In [19]:
const_stand_race = const_stand[const_stand['raceId'] == (current_raceid)]
const_stand_race = const_stand_race.sort_values(by='points', ascending=False)
const_stand_race[['name', 'points']].style.pipe(const_standings)

name,points
Ferrari,44.0
Mercedes,27.0
Haas F1 Team,10.0
Alfa Romeo,9.0
Alpine F1 Team,8.0
AlphaTauri,4.0
Red Bull,0.0
McLaren,0.0
Williams,0.0
Aston Martin,0.0


# Sources

- Shoutout to @VOPANI for creating this fantastic, clean, up to date F1 dataset
- In some ways inspired by this [[notebook](https://www.kaggle.com/code/ekrembayar/formula-1-70th-anniversary)] on the same dataset